# ------------------------------------------------------------
# Goal:
#   - Estimate the overall (train + test) treatment effect (ATE) using simple difference-in-means.
#   - Report ATE, standard error, and 95% confidence interval.
#   - Plot a Figure-2–style bar chart (as in the paper) showing control vs treatment means.
#
# If Y is continuous (e.g., revenue or sales amount):
#   - The same code works automatically; variances and SEs are computed using the sample variance of Y.
#   - The interpretation of ATE changes:
#       -> it becomes an average difference in Y units, not in probabilities.
# ------------------------------------------------------------

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# 95% normal quantile for CI computation
Z_95 = 1.96

def diff_in_means_se_ci(y, t, z=Z_95):
    """
    Compute difference-in-means ATE (treated - control), its SE, and CI.
    Works for both binary and continuous outcomes.

    Parameters
    ----------
    y : array-like
        Outcome variable (binary 0/1 or continuous values)
    t : array-like
        Treatment indicator (1 = treated, 0 = control)
    z : float
        z-score for confidence interval (1.96 = 95% CI)

    Returns
    -------
    ate : float
        Average treatment effect (mean(T=1) - mean(T=0))
    se_ate : float
        Standard error of the ATE
    ci : tuple
        (lower_bound, upper_bound) of the CI
    stats : dict
        Sample sizes, group means, and variances for diagnostics
    """

    # Ensure NumPy arrays and correct types
    y = np.asarray(y)
    t = np.asarray(t).astype(int)

    # Separate treated and control samples
    y1 = y[t == 1]
    y0 = y[t == 0]
    n1, n0 = len(y1), len(y0)

    if n1 == 0 or n0 == 0:
        raise ValueError("Need both treated and control observations to estimate ATE.")

    # Compute sample means
    m1, m0 = y1.mean(), y0.mean()

    # Compute unbiased sample variances
    v1, v0 = y1.var(ddof=1), y0.var(ddof=1)

    # Difference in means = simple ATE under random assignment
    ate = m1 - m0

    # Standard error of difference in means
    # Var(diff) = Var(Y1)/n1 + Var(Y0)/n0 (since groups are independent)
    se_ate = np.sqrt(v1 / n1 + v0 / n0)

    # 95% confidence interval using normal approximation
    ci_lo, ci_hi = ate - z * se_ate, ate + z * se_ate

    stats = dict(
        n_treat=n1,
        n_control=n0,
        mean_treat=m1,
        mean_control=m0,
        var_treat=v1,
        var_control=v0
    )

    return ate, se_ate, (ci_lo, ci_hi), stats


In [5]:
def mean_and_se(y):
    """
    Compute the mean and standard error of a sample.

    If Y is binary, this equals sqrt(p*(1-p)/n);
    if Y is continuous, it uses sample variance / n.

    Returns
    -------
    mean : float
    se : float
    """
    y = np.asarray(y)
    mean = y.mean()
    var = y.var(ddof=1)
    se = np.sqrt(var / len(y))
    return mean, se


In [6]:
if __name__ == "__main__":

    # === 1. Load training and test data, and combine them ===
    train = pd.read_csv("data/train_data.csv")
    test = pd.read_csv("data/test_data.csv")

    # Only keep treatment and outcome columns for this analysis
    required_cols = {"T", "Y"}
    if not required_cols.issubset(train.columns) or not required_cols.issubset(test.columns):
        raise ValueError("Expected columns 'T' and 'Y' in both train and test data.")

    # Combine train and test samples into one dataset
    df = pd.concat([train[["T", "Y"]], test[["T", "Y"]]], ignore_index=True)

    # === 2. Estimate overall treatment effect (ATE) ===
    ate, se_ate, (ci_lo, ci_hi), stats = diff_in_means_se_ci(df["Y"], df["T"])

    print("\n=== Overall (Train + Test) ===")
    print(f"n_control = {stats['n_control']}, n_treat = {stats['n_treat']}")
    print(f"Mean(Control)  = {stats['mean_control']:.4f}")
    print(f"Mean(Treatment)= {stats['mean_treat']:.4f}")
    print(f"ATE (T - C)    = {ate:.4f}")
    print(f"SE(ATE)        = {se_ate:.4f}")
    print(f"95% CI         = [{ci_lo:.4f}, {ci_hi:.4f}]")

    # === 3. Prepare group means and SEs for visualization ===

    # Make sure T is a flat 1-D Series, not a DataFrame
    df["T"] = np.ravel(df["T"])

    # Now the boolean masks will be 1-D
    y1 = df.loc[df["T"] == 1, "Y"].to_numpy()
    y0 = df.loc[df["T"] == 0, "Y"].to_numpy()

    m0, se0 = mean_and_se(y0)
    m1, se1 = mean_and_se(y1)


    # === 4. Plot Figure-2–style bar chart ===
    os.makedirs("plots", exist_ok=True)

    fig, ax = plt.subplots(figsize=(6, 4))

    groups = ["Control", "Treatment"]
    means = [m0, m1]
    # 95% CI for each group mean = mean ± 1.96 * SE
    cis = [Z_95 * se0, Z_95 * se1]

    # Draw bars with error bars (capsize for horizontal lines)
    ax.bar(groups, means, yerr=cis, capsize=6)

    # Labeling
    ax.set_ylabel("Mean outcome (Y)")
    ax.set_title("Overall Means with 95% CI — Combined")

    # Fix Y-axis range to focus on a realistic outcome window
    ax.set_ylim(0.28, 0.31)

    # Add light horizontal grid lines for readability
    ax.grid(True, axis="y", linestyle="--", alpha=0.5)

    plt.tight_layout()

    # Save figure
    outpath = "plots/figure2_overall_ate_combined_explained.png"
    plt.savefig(outpath, dpi=160)
    plt.close(fig)

    print(f"Saved plot → {outpath}")



=== Overall (Train + Test) ===
n_control = 24850, n_treat = 25150
Mean(Control)  = 0.3019
Mean(Treatment)= 0.2934
ATE (T - C)    = -0.0085
SE(ATE)        = 0.0041
95% CI         = [-0.0165, -0.0005]
Saved plot → plots/figure2_overall_ate_combined_explained.png
